# Navier-Stokes Animation

Navier-stokes are partial differential equations that describe the motion of viscous fluid substances, their simplified equation is defined as:

\begin{equation}
    \label{eq:navierstrokes}
    \frac{\partial u}{\partial t} + u \cdot \nabla u = g - \frac{1}{\rho} \nabla p + v \nabla ^2 u
\end{equation}

where:


- $u$ is the velocity vector, 
- $g$ is the acceleration vector due to a body force, 
- $p$ is pressure, 
- $v$ = $\frac{\mu}{\varrho}$ is the kinematic viscosity, 
- $\mu$ is the dynamic viscosity, and 
- $\rho$ is the density.


In this notebook we tried to do a simple 2D animation version using Python folowing:

https://www.dgp.toronto.edu/public_user/stam/reality/Research/pdf/GDC03.pdf
and 
https://thecodingtrain.com/challenges/132-fluid-simulation

### Code

imports

In [1]:
!python --version

Python 3.8.15


In [2]:
import pygame
import pygame.draw as g
import numpy as np
import math
import time

pygame 2.1.2 (SDL 2.0.18, Python 3.8.15)
Hello from the pygame community. https://www.pygame.org/contribute.html


constants to animate

In [3]:
# Constants
N = 64 # Matrix/field NxN
itera = 16 # num of iterations to the linear solve

In [4]:
# here if you want to make the fluid bounce in someting
def set_bnd(b, x, N):

    pass

#### Explanation  

in order to make the code runs faster, python needs to avoid loops, hence we use numpy to doa similar calculation but more fast.

So  this:
```python
for j in range(1, N-1):
    for i in range(1, N-1):
        x[i, j] = (x0[i, j] + a*(
            x[i+1, j] + x[i-1, j] +
            x[i, j+1] + x[i, j-1]
        )) * cRecip
```
Became this:
```python
matsum = x[2:, 1:-1] + x[:-2, 1:-1] + x[1:-1, 2:] + x[1:-1, :-2]
x[1:-1, 1:-1] = (x0[1:-1, 1:-1] + a*(matsum)) * cRecip
```
And this:
```python
for j in range(1, N - 1):
    for i in range(1, N - 1):
        velocX[i, j] -= 0.5 * (
            p[i+1, j] - p[i-1, j]
        )*N
        velocY[i, j] -= 0.5 * (
            p[i, j+1] - p[i, j-1]
        )*N
```
Became this:

```python
r1 = p[2:, 1:-1] - p[:-2, 1:-1]
r2 = p[1:-1, 2:] - p[1:-1, :-2]
velocX[1:-1, 1:-1] -= 0.5 * r1 * N
velocY[1:-1, 1:-1] -= 0.5 * r2 * N
```

## Linear solve

Iterative numerical method to solve part of the equation.

In [5]:
def lin_solve(b, x, x0, a, c):
    cRecip = 1.0 / c
    for t in range(itera):
        matsum = x[2:, 1:-1] + x[:-2, 1:-1] + x[1:-1, 2:] + x[1:-1, :-2]
        x[1:-1, 1:-1] = (x0[1:-1, 1:-1] + a*(matsum)) * cRecip
        set_bnd(b, x, N)

## Project 

Project the fluid acordly witn the vector field.
This operation runs through all the cells and fixes them up so everything is in equilibrium

In [6]:
def project(velocX, velocY, p, div):
    for j in range(1, N - 1):
        for i in range(1, N - 1):
            div[i, j] = (-0.5 * (
                velocX[i+1, j] - velocX[i-1, j] +
                velocY[i, j+1] - velocY[i, j-1])
            ) / N
            p[i, j] = 0

    set_bnd(0, div, N)
    set_bnd(0, p, N)
    lin_solve(0, p, div, 1, 6)

    r1 = p[2:, 1:-1] - p[:-2, 1:-1]
    r2 = p[1:-1, 2:] - p[1:-1, :-2]
    velocX[1:-1, 1:-1] -= 0.5 * r1 * N
    velocY[1:-1, 1:-1] -= 0.5 * r2 * N

    set_bnd(1, velocX, N)
    set_bnd(2, velocY, N)

# Advect

Move the things around.

In [7]:
def advect(b, d, d0, velocX, velocY, dt):
    dtx = dt * (N - 2)
    dty = dt * (N - 2)

    for j in range(1, N - 1):
        for i in range(1, N - 1):
            tmp1 = dtx * velocX[i, j]
            tmp2 = dty * velocY[i, j]

            x = float(i - tmp1)
            y = float(j - tmp2)

            if x < 0.5:
                x = 0.5
            if x > N - 1.5:
                x = N - 1.5
            i0 = math.floor(x)
            i1 = i0 + 1.0

            if y < 0.5:
                y = 0.5
            if y > N - 1.5:
                y = N - 1.5
            j0 = math.floor(y)
            j1 = j0 + 1.0

            s1 = x - i0
            s0 = 1.0 - s1
            t1 = y - j0
            t0 = 1.0 - t1

            i0i = int(i0)
            i1i = int(i1)
            j0i = int(j0)
            j1i = int(j1)

            d[i, j] = s0 * (t0 * d0[i0i, j0i]) + (t1 * d0[i0i, j1i])\
                    + s1 * (t0 * d0[i1i, j0i]) + (t1 * d0[i1i, j1i])

    set_bnd(b, d, N)

$dt$ changes per iteration, in viscosity $v$ diff in $\vec{x}$ based on $\vec{x}_0$

In [8]:
# NavierStokes
def diffuse(b, x, x0, diff, dt):
    a = dt * diff * (N - 2) * (N - 2)
    lin_solve(b, x, x0, a, 1 + 6 * a)


Fluid class, all the variables needed to run the animation are here.

In [11]:
class Fluid:

    def __init__(self, dt, diffusion, viscosity, squire_dim=10):
        self.squire_dim = squire_dim
        self.size = N
        self.dt = dt
        self.diff = diffusion
        self.visc = viscosity

        # Density prev/atual corante que vai no fluido
        self.s = np.zeros([N, N])
        self.density = np.zeros([N, N])

        # Velocity prev/atual
        self.Vx = np.zeros([N, N])
        self.Vy = np.zeros([N, N])

        self.Vx0 = np.zeros([N, N])
        self.Vy0 = np.zeros([N, N])

    def addDensity(self, x, y, amount):
        if 0 < x < N-1:
            if 0 < y < N-1:
                self.density[x, y] += amount
                return
        # print(x, y)

    def addVelocity(self, x, y, amountX, amountY):
        if 0 < x < N-1:
            if 0 < y < N-1:
                self.Vx[x, y] += amountX
                self.Vy[x, y] += amountY
                self.Vx0[x, y] += amountX
                self.Vy0[x, y] += amountY
                return

    def fadeDensity(self):
        self.density = np.clip(self.density - 0.02, 0, 255)

    def draw(self, screen, _x=0, _y=0):
        squire_dim = self.squire_dim
        color_fill = [0, 0, 0]
        for i in range(N):
            for j in range(N):
                d = self.density[i, j]/2
                d = max(d, 0)
                d = min(d, 255)
                color_fill[0] = d
                color_fill[1] = d
                color_fill[2] = d
                # print(self.density[i, j])
                g.rect(
                    screen,
                    color_fill,
                    [_x + i * squire_dim, _y + j * squire_dim,
                     squire_dim, squire_dim])

    def drawVel(self, screen, _x=0, _y=0):
        squire_dim = self.squire_dim
        color_fill = [255, 0, 0]
        for i in range(N):
            for j in range(N):
                x1 = i * squire_dim
                y1 = j * squire_dim
                vx = self.Vx[i, j]
                vy = self.Vy[i, j]
                if not (abs(vx) < 0.1 and abs(vy) <= 0.1):
                    g.line(screen, color_fill, [x1, y1], [x1 + vx*squire_dim, y1 + vy * squire_dim])

    def update(self):
        # diffuse the velocities based on time step and viscosity
        start = time.time()
        diffuse(1, self.Vx0, self.Vx, self.visc, self.dt)
        diffuse(2, self.Vy0, self.Vy, self.visc, self.dt)
        
        project(self.Vx0, self.Vy0, self.Vx, self.Vy)
        
        advect(1, self.Vx, self.Vx0, self.Vx0, self.Vy0, self.dt)
        advect(2, self.Vy, self.Vy0, self.Vx0, self.Vy0, self.dt)
        
        project(self.Vx, self.Vy, self.Vx0, self.Vy0)
        
        diffuse(0, self.s, self.density, self.diff, self.dt)
        advect(0, self.density, self.s, self.Vx, self.Vy, self.dt)
        end = time.time()
        # print("Time/iteration", end - start)


Main code run

comands:
 - A - start a new fluid with all fields 0
 - F - Stop to render the density
 - V - Stop to render the vectors
 - SPACE - Pause the animation

In [10]:
# initialize the pygame module
pygame.init()
pygame.display.set_caption("Navier stokes Anim")

screen = pygame.display.set_mode(
    (640, 640),
    pygame.RESIZABLE)

clock = pygame.time.Clock()
#------------------------------------
fluid = Fluid(0.25, 0.00025, 0.00025)
# fluid = Fluid(0.2, 0.1, 0.000001)

#------------------------------------
# main loop
running = True
mouse_pressed = False
omx, omy = 0, 0

flag_draw_fog = True
flag_draw_vec = True
flag_update = True
while running:
    g.rect(screen, [0, 0, 0], pygame.Rect(0, 0, 640, 640))
    # event handling, gets all event from the event queue
    mx, my = pygame.mouse.get_pos()
    for event in pygame.event.get():
        # only do something if the event is of type QUIT
        if event.type == pygame.QUIT:
            # change the value to False, to exit the main loop
            running = False

        if event.type == pygame.MOUSEBUTTONDOWN:
            mouse_pressed = True
            omx, omy = mx, my
        if event.type == pygame.MOUSEBUTTONUP:
            mouse_pressed = False
            
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_a:
                fluid = Fluid(0.25, 0.00025, 0.00025)
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_f:
                flag_draw_fog = False if flag_draw_fog else True
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_v:
                flag_draw_vec = False if flag_draw_vec else True
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                flag_update = False if flag_update else True
    
    if flag_update:
        fluid.update()
    if flag_draw_fog:
        fluid.draw(screen)
    if flag_draw_vec:
        fluid.drawVel(screen)
    fluid.addDensity(int(N/2), int(N/2), 2048)
    #fluid.fadeDensity()
    if mouse_pressed:
        # fluid.addDensity(mx // fluid.squire_dim, my // fluid.squire_dim, 100)
        fluid.addVelocity(mx // fluid.squire_dim, my // fluid.squire_dim, (mx - omx)/fluid.squire_dim,  (my - omy)/fluid.squire_dim)
        # fluid.addVelocity(omx // fluid.squire_dim, omy // fluid.squire_dim, (mx - omx)/fluid.squire_dim,  (my - omy)/fluid.squire_dim)
        # fluid.addVelocity(int(N/2), int(N/2), mx - omx,  my - omy)
    pygame.display.flip()
    # Limit the frame rate
    clock.tick(30)

pygame.quit()